In [156]:
import pandas as pd
from xml.dom import minidom
import drawSvg as draw
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import os
import pathlib

In [166]:
class Convertor():
    def convert(self, file, loc, padding = 50):
        data = minidom.parse(file)
        points = data.getElementsByTagName('point')
        strokes = data.getElementsByTagName('stroke')
        
        # creat dictionary for id -> point
        point_dic = {}
        axis = [1e6, 0, 1e6, 0]
        for el in points:
            x = el.attributes['x'].value
            axis[0] = min(axis[0],int(x))
            axis[1] = max(axis[1],int(x))
            y = el.attributes['y'].value
            axis[2] = min(axis[2],int(y))
            axis[3] = max(axis[3],int(y))
            time = el.attributes['time'].value
            point_dic[el.attributes['id'].value] = (x ,y , time);
        axis = [axis[0]-padding, axis[1]+padding, axis[2]-padding, axis[3]+padding]
        plt.axis(axis)
        
        # extract strokes
        for st in strokes:
            pts = st.getElementsByTagName('arg')
            x = [int(point_dic[pt.firstChild.nodeValue][0]) for pt in pts]
            y = [int(point_dic[pt.firstChild.nodeValue][1]) for pt in pts]
            plt.axis(axis)
            plt.plot(x,y)
        
        #save image
        plt.savefig(loc)
        plt.close()
        
    
    def explore(self, directory, save):
        if not os.path.exists(save):
            os.mkdir(save)
        for path in pathlib.Path(directory).iterdir():
            if path.is_dir() and str(path) != save:
                folder = os.path.basename(path)
                self.explore(path, os.path.join(save, folder))
            elif path.is_file():
                file = os.path.basename(path)
                if file.endswith(".xml"):
                    #print(os.path.join(save, str(file[:-4] + '.png')))
                    dis = str(os.path.join(save, file[:-4]))
                    if dis != "": 
                        try:
                            self.convert(str(path), dis)
                        except:
                            print("could not convert file" + file)
                        

In [167]:
conv = Convertor()
directory = "../ASIST_Dataset/Data/Data_A"
save = os.path.join(directory + "/images")
conv.explore(directory, save)

could not convert file11_4aad22e3-4075-4871-b5e5-30c933078a31.xml
